## Khan Academy Math Student Dashboard: Data Cleaning
We will be creating a student dashboard that displays each student's progress through Khan Academy Math. The goal is to showcase:
- How the student is performing overall and this month?
- What is their assignment completion rate?
- Which topics are students strongest and weakest in?
- The students' percentile in the class?
- Is the student growing?

We want to automate the dashboarding process for each class in Lyceum Village, grades K-8. We will be testing on grade K-1 students' Khan Academy Math first to create functions and a webpage to showcase their progress.

### Import the Data
The data imported is for grade K-1. The students' names are already adjusted to an alias name.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../Resources/K_1-Math-Anonymous.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Assignment Name              266 non-null    object 
 1   Student Name                 266 non-null    object 
 2   Score At Due Date            26 non-null     float64
 3   Score Best Ever              30 non-null     float64
 4   Points Possible              30 non-null     float64
 5   Number Of Attempts           266 non-null    int64  
 6   Most Recent Completion Date  30 non-null     object 
 7   Start Date                   266 non-null    object 
 8   Due Date                     266 non-null    object 
 9   Assignment URL               266 non-null    object 
 10  Assignment Type              266 non-null    object 
 11  Due Date (no time)           266 non-null    object 
dtypes: float64(3), int64(1), object(8)
memory usage: 25.1+ KB


Our csv dataset will have 11 variables.

Types of data:
- 3 float: 
    - Score At Due Date
    - Score Best Ever
    - Points Possible
- 1 integer:
    - Number Of Attempts
- 7 string:
    - Assignment Name
    - Student Name
    - Most Recent Completion Date
    - Start Date
    - Due Date
    - Assignment URL
    - Assignment Type

The data type so far matches what they should be.

There are 266 rows total. There are missing data in the following columns:
- Score At Due Date: 240 missing
- Score Best Ever: 224 missing
- Points Possible: 224 missing
- Most Recent Completion Date: 224 missing

They are missing data because some students did not complete their assignments. We will be removing these data in actual student analysis.

### Student Anonymity
We will need to create a student ID column to give each student anonymity on their scores when it is being displayed to other parents.
Their ID's will be formatted as so: grade_studentinitial_number <br>

Example: G2_KL_1

#### Create Student ID
We will be automating this system by having it done automatically only once. The application will request the grade level from the user and store the student ID information into a PostgreSQL database.

In [5]:
df['Student Name'].unique()

array(['Ned Ethans', 'Anna Kite', 'April Luna', 'Andy Hong',
       'Abby Nguyen', 'Everett Chase', 'Kyle Anderson', 'Kevin Martelle',
       'Misa Bing', 'Olsen Le', 'Terry Long'], dtype=object)

In [3]:
# User input grade level
stu_grade = input("Enter the student grade level: ")

In [4]:
# Loop through the list of unique students and create their ID
stu_id_list = []
for name in df['Student Name']:
    stu_name = name.split(" ")
    stu_id = "G" + stu_grade + "_" + stu_name[0][0] + stu_name[1][0]
    stu_id_list.append(stu_id)
stu_id_list[:5]

['GK1_NE', 'GK1_AK', 'GK1_AL', 'GK1_AH', 'GK1_AN']

In [5]:
# Add in the numeric value to account for students with the same initials
stu_id_list2 = []
if len(df['Student Name']) == len(stu_id_list):
    for id in stu_id_list:
        # Add 1 at the end because there are no repeated initials
        stu_id_list2.append(id + "1")
else:
    # change the number depending on the student name
    # current list of students do not have repeated initials. Create a dummy list to test this function on
    stu_id_list2
stu_id_list2[:5]

['GK1_NE1', 'GK1_AK1', 'GK1_AL1', 'GK1_AH1', 'GK1_AN1']

In [6]:
# Add id column to the front of the dataframe
df.insert(loc=0, column='Student ID', value=stu_id_list2)
df.head()

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time)
0,GK1_NE1,Comparing numbers to 10,Ned Ethans,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:50PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,"
1,GK1_AK1,Add within 10,Anna Kite,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,"
2,GK1_AL1,Add within 10,April Luna,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,"
3,GK1_AH1,Add within 10,Andy Hong,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,"
4,GK1_AN1,Add within 10,Abby Nguyen,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,"


## Missing Data
NaN in the Score At Due Date, Score Best Ever, and Points Possible columns means that students did not complete their assignment. Thus, we will need to create a column that states the completion of their assignment.

Create a column with these values:
- Missing: Has 0 as the Number Of Attempts
- Complete: Has 1 or higher Number of Attempts and the Most Recent Completion Date is before or on the Due Date
- Late: Has 1 or more Number Of Attempts and the Most Recent Completion Date is past the Due Date

### Convert Date Columns to Date type
Convert the two columns from string object to date:
- Most Recent Completion Date
- Due Date

In [7]:
# Find the rows with no missing data
df_val = df[df['Most Recent Completion Date'].isna()==False]
df_val.head()

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time)
96,GK1_KM1,Counting: Unit test,Kevin Martelle,13.0,13.0,13.0,2,"Sep 20th, 4:55PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/early-math/cc...,Unit Test,"Sep 24th,"
100,GK1_TL1,Counting: Unit test,Terry Long,13.0,13.0,13.0,1,"Sep 20th, 4:42PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/early-math/cc...,Unit Test,"Sep 24th,"
107,GK1_KM1,Add within 5,Kevin Martelle,7.0,7.0,7.0,1,"Sep 20th, 5:01PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Sep 24th,"
111,GK1_TL1,Add within 5,Terry Long,7.0,7.0,7.0,1,"Sep 20th, 4:50PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Sep 24th,"
118,GK1_KM1,Subtract within 5,Kevin Martelle,7.0,7.0,7.0,1,"Sep 20th, 5:04PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Sep 24th,"


In [9]:
# Function to convert the date from string type to date type
import datetime
import re
def convertDate(date_str, year):
    split_date = re.split(", | ", date_str) # Splits the date into Month, Day, Time
    day_num = int(re.findall('\d+', split_date[1])[0]) # Removes letters from the day. Example: 8th becomes 8
    month_num = datetime.datetime.strptime(split_date[0], '%b').month # Converts the month into the month's number
    
    # we are missing the year; we will have to have the user input it
    # Combined date
    date_str = str(month_num) + "/" + str(day_num) + "/" + year + " " + split_date[2]
    comb_date = datetime.datetime.strptime(date_str, '%m/%d/%Y %H:%M%p')

    return comb_date
    

In [10]:
progress_list = []
index_attempts = df.columns.get_loc('Number Of Attempts')
index_duedate = df.columns.get_loc('Due Date')
index_compdate = df.columns.get_loc('Most Recent Completion Date')
report_year = input('Year of report: ')
for i, r in df.iterrows():
    if r[index_attempts] == 0:
        progress_list.append('Missing')
    else:
        duedate_formatted = convertDate(r[index_duedate], report_year)
        compdate_formatted = convertDate(r[index_compdate], report_year)
        if duedate_formatted > compdate_formatted:
            progress_list.append('Complete')
        else:
            progress_list.append('Late')

In [11]:
# Add Progress column into the df
df['Progress'] = progress_list
df.head()

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time),Progress
0,GK1_NE1,Comparing numbers to 10,Ned Ethans,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:50PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,",Missing
1,GK1_AK1,Add within 10,Anna Kite,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,",Missing
2,GK1_AL1,Add within 10,April Luna,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,",Missing
3,GK1_AH1,Add within 10,Andy Hong,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,",Missing
4,GK1_AN1,Add within 10,Abby Nguyen,NaN,NaN,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,",Missing


The Progress column which indicates if the assignment is missing, late, or complete. Now we can convert all missing data from the following columns into 0 if they are marked "Missing":
- Score At Due Date
- Score Best Ever

In [12]:
df.loc[df['Progress'] == 'Missing', ['Score At Due Date', 'Score Best Ever']] = 0
df.head()

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time),Progress
0,GK1_NE1,Comparing numbers to 10,Ned Ethans,0.0,0.0,NaN,0,NaN,"Sep 29th, 4:50PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,",Missing
1,GK1_AK1,Add within 10,Anna Kite,0.0,0.0,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,",Missing
2,GK1_AL1,Add within 10,April Luna,0.0,0.0,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,",Missing
3,GK1_AH1,Add within 10,Andy Hong,0.0,0.0,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,",Missing
4,GK1_AN1,Add within 10,Abby Nguyen,0.0,0.0,NaN,0,NaN,"Sep 29th, 4:48PM","Oct 8th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Oct 8th,",Missing


The following columns contains missing data for missed assignments, but cannot be set to 0:
- Points Possible: this is the total points received for the assignment. It should be set depending on the points available per assignment.
- Most Recent Completion Date: this will be left as missing as it is unrelated to scores

We need to find the Points Possible value for each assignment, and fill in the missing information.

In [13]:
df_no_missing = df.loc[df['Points Possible'].isna() == False]
df_no_missing.head()

,Student ID,Assignment Name,Student Name,Score At Due Date,Score Best Ever,Points Possible,Number Of Attempts,Most Recent Completion Date,Start Date,Due Date,Assignment URL,Assignment Type,Due Date (no time),Progress
96,GK1_KM1,Counting: Unit test,Kevin Martelle,13.0,13.0,13.0,2,"Sep 20th, 4:55PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/early-math/cc...,Unit Test,"Sep 24th,",Complete
100,GK1_TL1,Counting: Unit test,Terry Long,13.0,13.0,13.0,1,"Sep 20th, 4:42PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/early-math/cc...,Unit Test,"Sep 24th,",Complete
107,GK1_KM1,Add within 5,Kevin Martelle,7.0,7.0,7.0,1,"Sep 20th, 5:01PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Sep 24th,",Complete
111,GK1_TL1,Add within 5,Terry Long,7.0,7.0,7.0,1,"Sep 20th, 4:50PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Sep 24th,",Complete
118,GK1_KM1,Subtract within 5,Kevin Martelle,7.0,7.0,7.0,1,"Sep 20th, 5:04PM","Sep 10th, 10:33AM","Sep 24th, 11:59PM",https://www.khanacademy.org/math/cc-kindergart...,Exercise,"Sep 24th,",Complete


In [20]:
# Check if the assignments in the dataframe with no missing values is the same for those that are missing.
df_nm_uniq = df.loc[df['Points Possible'].isna() == False, ['Assignment Name']]['Assignment Name'].unique()
df_nm_uniq

array(['Counting: Unit test', 'Add within 5', 'Subtract within 5',
       'Count objects 2', 'Compare numbers of objects 1',
       'Comparing numbers to 10', 'Counting: Quiz 2', 'Numbers to 100',
       'Numbers to 120', 'Count tens', 'Counting: Quiz 1',
       'Count with small numbers', 'Count in order',
       'Find 1 more or 1 less than a number', 'Missing numbers'],
      dtype=object)

In [19]:
# List of unique Assignment Names from values with missing data
df_m_uniq = df.loc[df['Points Possible'].isna() == True, ['Assignment Name']]['Assignment Name'].unique()
df_m_uniq

array(['Comparing numbers to 10', 'Add within 10', 'Subtract within 10',
       'Addition and subtraction intro: Quiz 2',
       'Relate addition and subtraction', 'Making 5',
       'Making small numbers in different ways',
       'Addition and subtraction intro: Quiz 1',
       'Make 10 (grids and number bonds)', 'Counting: Unit test',
       'Add within 5', 'Subtract within 5', 'Count objects 2',
       'Compare numbers of objects 1', 'Compare numbers of objects 2',
       'Counting: Quiz 2', 'Numbers to 100', 'Numbers to 120',
       'Count tens', 'Counting: Quiz 1', 'Count with small numbers',
       'Count in order', 'Find 1 more or 1 less than a number',
       'Missing numbers'], dtype=object)